In [4]:
import requests
zipurl = 'https://mtgjson.com/api/v5/AllPrintings.json.zip'
resp = requests.get(zipurl, verify=False) # verify=False on work computer bc IT proxy / vpn that's in place DO NOT recommend in general

zname = "data/tmp/AllPrintings.json.zip"
zfile = open(zname, 'wb')
zfile.write(resp.content)
zfile.close()

from zipfile import ZipFile

# Create a ZipFile Object and load sample.zip in it
with ZipFile('data/tmp/AllPrintings.json.zip', 'r') as zipObj:
   # Extract all the contents of zip file in different directory
   zipObj.extractall('data/raw')

In [18]:
import pandas as pd
import json

with open('data/raw/AllPrintings.json', encoding="utf8") as f:
    data = json.load(f)

editions = data['data'].keys()

In [54]:
set_info = []
card_info = []

for edition in editions:
    set_name = data['data'][edition].get('name')
    set_code = data['data'][edition].get('code')
    cards_base_n = data['data'][edition].get('baseSetSize')
    cards_n = data['data'][edition].get('totalSetSize')
    calc_n = len(data['data'][edition]['cards']) #add in check if the abs(calc_n - cards_n) <> 0
    release_date = data['data'][edition].get('releaseDate')

    tmp_set_info = {
        'set_name': set_name,
        'set_code': set_code,
        'release_date': release_date,
        'base_set_size': cards_base_n,
        'cards_total_n': cards_n,
        'cards_total_n_calc': calc_n,
        'type': data['data'][edition].get('type'),
        'isOnlineOnly': data['data'][edition].get('isOnlineOnly')
    }

    set_info.append(tmp_set_info)
    #print('Number of cards:', cards_n)

    for i in range(1,cards_n):
        # set basic json structure we're going iterate through
        json_structure_cards = data['data'][edition]['cards'][i]

        # build out our tmp single card dictionary
        tmp_card_info = {
            'set_name': set_name,
            'card_name': json_structure_cards.get('name'),
            'mtgjson_id': json_structure_cards['identifiers'].get('mtgjsonV4Id'),
            'colors': json_structure_cards.get('colors'),
            'hasNonFoil': json_structure_cards.get('hasNonFoil'),
            'hasFoil': json_structure_cards.get('hasFoil'),
            'isReprint': json_structure_cards.get('isReprint')
        }

        # append tmp single card dictionary to the growing all cards dictionary
        card_info.append(tmp_card_info)

    # clean up tmp dicts
    del tmp_set_info
    del tmp_card_info

    # break the loop to just grab one set for now
    break

# dump our card dictionary into a pandas d
set_info_df = pd.DataFrame.from_dict(set_info)
card_info_df = pd.DataFrame.from_dict(card_info)


In [55]:
card_info_df

,set_name,card_name,mtgjson_id,colors,hasNonFoil,hasFoil,isReprint
0,Tenth Edition,Angel of Mercy,9eb2e54c-a12b-5e88-a9c0-d8c84c52d59c,[W],True,False,True
1,Tenth Edition,Aven Cloudchaser,c0de6fd8-367a-50fb-b2f4-2b8fa5aeb7d9,[W],True,False,True
2,Tenth Edition,Ballista Squad,bfbb65b1-f1bb-5355-9495-fb094f9b0782,[W],True,True,True
3,Tenth Edition,Bandage,a634f6df-fd74-54d9-a56c-3fdd2ad3e9bf,[W],True,True,True
4,Tenth Edition,Beacon of Immortality,43f8222f-3fd5-5491-9e76-5b1cc9a86e93,[W],True,True,True
...,...,...,...,...,...,...,...
502,Tenth Edition,Fog Elemental,6cbe5d76-5244-5a2a-b9d1-30272b9e7d7a,[U],False,True,True
503,Tenth Edition,Mahamoti Djinn,9b06de27-31d8-503b-ae35-b291ea9da383,[U],False,True,True
504,Tenth Edition,March of the Machines,cfb2cdd2-2a70-5ec8-b4d0-bd9d51c2db75,[U],False,True,True
505,Tenth Edition,Mind Bend,9ce04474-1ee6-5ac2-812a-dace4c7331ce,[U],False,True,True
